In [1]:
import cv2
import mediapipe as mp
import numpy as np
from collections import OrderedDict

In [15]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Define adjustable thresholds (can be edited easily)
THRESHOLDS = {
    "front_knee_angle": (70, 120),
    "back_leg_angle": (150, 180),
    "hip_orientation": (0, 30),
    "torso_angle": (0, 100),
    "arm_angle": (160, 180),
    "shoulder_hip_alignment": (0, 40)
}

# Function to calculate angle between three points
def calculate_angle(p1, p2, p3):
    a = np.array(p1)  # First point
    b = np.array(p2)  # Vertex
    c = np.array(p3)  # Third point
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle) * 180 / np.pi
    return angle

# Function to check pose and return errors
def check_warrior_pose(landmarks):
    errors = []
    
    # Extract key landmarks (x, y coordinates)
    l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y]
    r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y]
    l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y]
    r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y]
    l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y]
    r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y]
    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y]
    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y]
    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y]

    # Dynamically determine front and back legs based on knee angle
    left_knee_angle = calculate_angle(l_hip, l_knee, l_ankle)
    right_knee_angle = calculate_angle(r_hip, r_knee, r_ankle)
    
    if left_knee_angle < right_knee_angle:  # Left leg is front leg
        front_hip, front_knee, front_ankle = l_hip, l_knee, l_ankle
        back_hip, back_knee, back_ankle = r_hip, r_knee, r_ankle
        front_knee_angle = left_knee_angle
        back_leg_angle = right_knee_angle
        hip_angle = calculate_angle(l_hip, r_hip, [r_hip[0] + 1, r_hip[1]])  # Left to right (rightward)
    else:  # Right leg is front leg
        front_hip, front_knee, front_ankle = r_hip, r_knee, r_ankle
        back_hip, back_knee, back_ankle = l_hip, l_knee, l_ankle
        front_knee_angle = right_knee_angle
        back_leg_angle = left_knee_angle
        hip_angle = calculate_angle(r_hip, l_hip, [l_hip[0] - 1, l_hip[1]])  # Right to left (leftward)

    # Calculate other angles
    torso_angle = calculate_angle(
        [(l_hip[0] + r_hip[0]) / 2, (l_hip[1] + r_hip[1]) / 2],
        [(l_shoulder[0] + r_shoulder[0]) / 2, (l_shoulder[1] + r_shoulder[1]) / 2],
        [0, 1]  # Vertical reference
    )
    l_arm_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
    r_arm_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
    shoulder_hip_angle = calculate_angle(l_shoulder, r_shoulder, r_hip)

    # Check thresholds and collect errors
    if not THRESHOLDS["front_knee_angle"][0] <= front_knee_angle <= THRESHOLDS["front_knee_angle"][1]:
        errors.append("Bend your front knee more" if front_knee_angle > 100 else "Straighten your front knee slightly")
    if not THRESHOLDS["back_leg_angle"][0] <= back_leg_angle <= THRESHOLDS["back_leg_angle"][1]:
        errors.append("Straighten your back leg" if back_leg_angle < 160 else "Relax your back leg slightly")
    
    # Hip orientation check (dynamic based on front leg)
    if not THRESHOLDS["hip_orientation"][0] <= hip_angle <= THRESHOLDS["hip_orientation"][1]:
        if front_hip == r_hip:  # Right leg is front
            if r_hip[1] > l_hip[1]:  # Right hip higher
                errors.append("Level your hips; right hip is too high")
            else:  # Left hip higher
                errors.append("Level your hips; left hip is too high")
        else:  # Left leg is front
            if l_hip[1] > r_hip[1]:  # Left hip higher
                errors.append("Level your hips; left hip is too high")
            else:  # Right hip higher
                errors.append("Level your hips; right hip is too high")


    
    if not THRESHOLDS["arm_angle"][0] <= l_arm_angle <= THRESHOLDS["arm_angle"][1] or \
       not THRESHOLDS["arm_angle"][0] <= r_arm_angle <= THRESHOLDS["arm_angle"][1]:
        errors.append("Raise your arms to shoulder level" if l_arm_angle < 170 or r_arm_angle < 170 else "Extend your arms fully")                

    #if not THRESHOLDS["torso_angle"][0] <= torso_angle <= THRESHOLDS["torso_angle"][1]:
    #    errors.append("Straighten your torso" if torso_angle < 80 else "Lean back slightly")
    #if not THRESHOLDS["shoulder_hip_alignment"][0] <= shoulder_hip_angle <= THRESHOLDS["shoulder_hip_alignment"][1]:
    #    errors.append("Align your shoulders over your hips")

    return errors[:3]  # Return top 3 errors


# Main loop
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame with MediaPipe
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if results.pose_landmarks:
        # Draw current skeleton
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Check pose and get errors
        errors = check_warrior_pose(results.pose_landmarks.landmark)

        # Display top 3 errors
        for i, error in enumerate(errors):
            cv2.putText(frame, error, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Warrior Pose Correction", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pose.close()

# Final

In [1]:
import cv2
import mediapipe as mp
import numpy as np
from collections import defaultdict
import time

In [6]:
class WarriorPoseAnalyzer:
    def __init__(self, record_seconds=10, fps=30):
        # Initialize MediaPipe Pose
        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils
        self.pose = self.mp_pose.Pose()

        # Define adjustable thresholds
        self.THRESHOLDS = {
            "front_knee_angle": (70, 120),
            "back_leg_angle": (150, 180),
            "hip_orientation": (0, 30),
            "torso_angle": (0, 100),
            "arm_angle": (160, 180),
            "shoulder_hip_alignment": (0, 40)
        }

        # Recording settings
        self.fps = fps
        self.delay_frames = 3 * fps  # 3 seconds delay
        self.record_frames = record_seconds * fps  # Number of frames to record
        self.frame_count = 0
        self.recording = False
        self.report = {
            "good_form_frames": 0,
            "error_counts": defaultdict(int)  # Tracks each error's frequency
        }

    def calculate_angle(self, p1, p2, p3):
        """Calculate the angle between three points in degrees."""
        a = np.array(p1)
        b = np.array(p2)
        c = np.array(p3)
        ba = a - b
        bc = c - b
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle) * 180 / np.pi
        return angle

    def check_warrior_pose(self, landmarks):
        """Analyze Warrior II pose and return top 3 errors."""
        errors = []

        l_hip = [landmarks[self.mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[self.mp_pose.PoseLandmark.LEFT_HIP].y]
        r_hip = [landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP].y]
        l_knee = [landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE].y]
        r_knee = [landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE].y]
        l_ankle = [landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE].y]
        r_ankle = [landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE].y]
        l_shoulder = [landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER].y]
        r_shoulder = [landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
        l_elbow = [landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW].x, landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW].y]
        r_elbow = [landmarks[self.mp_pose.PoseLandmark.RIGHT_ELBOW].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_ELBOW].y]
        l_wrist = [landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST].x, landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST].y]
        r_wrist = [landmarks[self.mp_pose.PoseLandmark.RIGHT_WRIST].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_WRIST].y]

        left_knee_angle = self.calculate_angle(l_hip, l_knee, l_ankle)
        right_knee_angle = self.calculate_angle(r_hip, r_knee, r_ankle)
        
        if left_knee_angle < right_knee_angle:
            front_hip, front_knee, front_ankle = l_hip, l_knee, l_ankle
            back_hip, back_knee, back_ankle = r_hip, r_knee, r_ankle
            front_knee_angle = left_knee_angle
            back_leg_angle = right_knee_angle
            hip_angle = self.calculate_angle(l_hip, r_hip, [r_hip[0] + 1, r_hip[1]])
        else:
            front_hip, front_knee, front_ankle = r_hip, r_knee, r_ankle
            back_hip, back_knee, back_ankle = l_hip, l_knee, l_ankle
            front_knee_angle = right_knee_angle
            back_leg_angle = left_knee_angle
            hip_angle = self.calculate_angle(r_hip, l_hip, [l_hip[0] - 1, l_hip[1]])

        torso_angle = self.calculate_angle(
            [(l_hip[0] + r_hip[0]) / 2, (l_hip[1] + r_hip[1]) / 2],
            [(l_shoulder[0] + r_shoulder[0]) / 2, (l_shoulder[1] + r_shoulder[1]) / 2],
            [0, 1]
        )
        l_arm_angle = self.calculate_angle(l_shoulder, l_elbow, l_wrist)
        r_arm_angle = self.calculate_angle(r_shoulder, r_elbow, r_wrist)
        shoulder_hip_angle = self.calculate_angle(l_shoulder, r_shoulder, r_hip)

        if not self.THRESHOLDS["front_knee_angle"][0] <= front_knee_angle <= self.THRESHOLDS["front_knee_angle"][1]:
            errors.append("Bend your front knee more" if front_knee_angle > 100 else "Straighten your front knee slightly")
        if not self.THRESHOLDS["back_leg_angle"][0] <= back_leg_angle <= self.THRESHOLDS["back_leg_angle"][1]:
            errors.append("Straighten your back leg" if back_leg_angle < 160 else "Relax your back leg slightly")
        
        if not self.THRESHOLDS["hip_orientation"][0] <= hip_angle <= self.THRESHOLDS["hip_orientation"][1]:
            if front_hip == r_hip:
                if r_hip[1] > l_hip[1]:
                    errors.append("Level your hips; right hip is too high")
                else:
                    errors.append("Level your hips; left hip is too high")
            else:
                if l_hip[1] > r_hip[1]:
                    errors.append("Level your hips; left hip is too high")
                else:
                    errors.append("Level your hips; right hip is too high")

        if not self.THRESHOLDS["arm_angle"][0] <= l_arm_angle <= self.THRESHOLDS["arm_angle"][1] or \
           not self.THRESHOLDS["arm_angle"][0] <= r_arm_angle <= self.THRESHOLDS["arm_angle"][1]:
            errors.append("Raise your arms to shoulder level" if l_arm_angle < 170 or r_arm_angle < 170 else "Extend your arms fully")

        return errors[:3]

    def process_frame(self, frame):
        """Process a single frame and return the annotated frame with errors or correct form message."""
        results = self.pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        if results.pose_landmarks:
            self.mp_drawing.draw_landmarks(frame, results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS)
            errors = self.check_warrior_pose(results.pose_landmarks.landmark)

            # Update frame count and recording logic
            self.frame_count += 1
            if self.frame_count > self.delay_frames and not self.recording:
                self.recording = True
                self.start_frame = self.frame_count
            if self.recording and (self.frame_count - self.start_frame) <= self.record_frames:
                if not errors:
                    self.report["good_form_frames"] += 1
                for error in errors:
                    self.report["error_counts"][error] += 1

            # Display errors or "Correct Form"
            if errors:
                for i, error in enumerate(errors):
                    cv2.putText(frame, error, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "Correct Form", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        return frame

    def generate_report(self):
        """Generate and print an exercise report."""
        total_recorded_frames = self.record_frames
        good_form_seconds = self.report["good_form_frames"] / self.fps
        total_seconds = total_recorded_frames / self.fps

        print("\n--- Warrior II Exercise Report ---")
        print(f"Total Recorded Time: {total_seconds:.2f} seconds")
        print(f"Good Form Duration: {good_form_seconds:.2f} seconds ({(good_form_seconds / total_seconds) * 100:.1f}%)")
        print("Errors Detected:")
        if self.report["error_counts"]:
            for error, count in self.report["error_counts"].items():
                error_seconds = count / self.fps
                print(f"  - '{error}': {count} frames ({error_seconds:.2f} seconds, {(count / total_recorded_frames) * 100:.1f}%)")
        else:
            print("  - No errors detected!")
        print("--------------------------------\n")

    def run(self):
        """Run the analyzer with webcam input."""
        cap = cv2.VideoCapture(0)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = self.process_frame(frame)
            cv2.imshow("Warrior Pose Correction", frame)

            # Stop recording and generate report when time is up
            if self.recording and (self.frame_count - self.start_frame) > self.record_frames:
                self.generate_report()
                break

            if cv2.waitKey(1) & 0xFF == ord('q'):
                if self.recording:
                    self.generate_report()
                break

        cap.release()
        cv2.destroyAllWindows()
        self.pose.close()

    def __del__(self):
        self.pose.close()


In [8]:
# Usage
if __name__ == "__main__":
    analyzer = WarriorPoseAnalyzer(record_seconds=10)  # Record for 10 seconds after 3-second delay
    analyzer.run()

<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_4376\918094476.py:5: SyntaxWarning: invalid escape sequence '\D'
  analyzer.process_image("./New folder\DATASET/TRAIN/warrior2/00000131.jpg", "output.jpg")
Exception ignored in: <function WarriorPoseAnalyzer.__del__ at 0x000001D569A1C7C0>
Traceback (most recent call last):
  File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_4376\1102365766.py", line 219, in __del__
  File "D:\anaconda\envs\fypvis\Lib\site-packages\mediapipe\python\solution_base.py", line 361, in close
    raise ValueError('Closing SolutionBase._graph which is already None')
ValueError: Closing SolutionBase._graph which is already None


Processed image saved to output.jpg


In [ ]:
# backend/main.py
from fastapi import FastAPI, WebSocket
import cv2
import asyncio
import base64
import numpy as np
import websockets
import json

app = FastAPI()

# Your existing exercise classes
from exercise1 import Exercise1Analyzer
from exercise2 import Exercise2Analyzer
from exercise3 import Exercise3Analyzer
from exercise4 import Exercise4Analyzer

# Initialize analyzers
analyzers = {
    "Squat": SquatAnalyzer(),
    "exercise2": Exercise2Analyzer(),
    "exercise3": Exercise3Analyzer(),
    "exercise4": Exercise4Analyzer()
}

class VideoProcessor:
    def __init__(self):
        self.cap = None
        self.is_running = False
        self.current_exercise = None

    async def process_video(self, websocket):
        if not self.cap:
            self.cap = cv2.VideoCapture(0)
        
        while self.is_running and self.current_exercise:
            ret, frame = self.cap.read()
            if not ret:
                break

            analyzer = analyzers[self.current_exercise]
            processed_frame, metrics = analyzer.process_frame(frame)
            
            _, buffer = cv2.imencode('.jpg', processed_frame)
            frame_base64 = base64.b64encode(buffer).decode('utf-8')
            
            await websocket.send(json.dumps({
                "frame": frame_base64,
                "metrics": metrics
            }))
            await asyncio.sleep(0.033)

    def stop(self):
        self.is_running = False
        if self.cap:
            self.cap.release()

video_processor = VideoProcessor()

async def websocket_handler(websocket):
    try:
        while True:
            data = json.loads(await websocket.recv())
            exercise = data.get("exercise")
            
            if exercise == "stop":
                video_processor.stop()
            elif exercise in analyzers:
                video_processor.current_exercise = exercise
                if not video_processor.is_running:
                    video_processor.is_running = True
                    asyncio.create_task(video_processor.process_video(websocket))
                    
    except websockets.ConnectionClosed:
        video_processor.stop()
    except Exception as e:
        video_processor.stop()

async def main():
    async with websockets.serve(
        websocket_handler,
        "localhost",
        8000
    ):
        await asyncio.Future()  # Run forever

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
# backend/main.py
import cv2
import asyncio
import json
import logging
import threading
import time
import websockets
from functools import partial

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Import your analyzer classes
"""""
IMPORTANT, ADD PATH
"""""
from squats_analyzer import SquatsAnalyzer

class VideoServer:
    def __init__(self):
        self.cap = None
        self.clients = set()
        self.event_loop = None
        self.server = None
        self.running = False
        self.current_analyzer = None

        # Initialize analyzers with a reference to the broadcast method
        self.analyzers = {
            "Squats": SquatsAnalyzer(),
        }

    async def process_frames(self, input_source=0):
        """Centralized frame processing loop."""
        self.cap = cv2.VideoCapture(input_source)
        if not self.cap.isOpened():
            logger.error(f"Error: Could not open video source {input_source}")
            return

        try:
            while self.running and self.cap.isOpened():
                start_time = time.time()

                success, frame = self.cap.read()
                if not success:
                    logger.info("End of video")
                    break

                # Process frame with the current analyzer if selected
                if self.current_analyzer:
                    processed_data = await self.current_analyzer.process_video(frame)
                    if processed_data:
                        await self._broadcast(processed_data)

                # Maintain ~30 FPS
                processing_time = time.time() - start_time
                await asyncio.sleep(max(0, 0.03 - processing_time))

        except Exception as e:
            logger.error(f"Error during frame processing: {e}")
        finally:
            if self.cap:
                self.cap.release()
            if self.current_analyzer:
                report = self.current_analyzer.generate_report()
                print("\n" + report)
                with open('report.txt', 'w') as f:
                    f.write(report)
            logger.info("Video processing stopped")

    async def _broadcast(self, message):
        """Broadcast a message to all connected clients."""
        if not self.clients:
            return
        message_json = json.dumps(message)
        await asyncio.gather(
            *[client.send(message_json) for client in self.clients],
            return_exceptions=True
        )

    def broadcast_message(self, message):
        """Broadcast a message to all connected clients from any thread."""
        if self.event_loop and self.clients:
            asyncio.run_coroutine_threadsafe(self._broadcast(message), self.event_loop)

    async def websocket_handler(self, websocket):
        self.clients.add(websocket)
        logger.info(f"New client connected: {websocket.remote_address}")
        try:
            async for message in websocket:
                try:
                    data = json.loads(message)
                    action = data.get('action')
                    exercise = data.get('exercise')
                    logger.info(f"Received action: {action}, exercise: {exercise}")

                    if action == 'connect':
                        await websocket.send(json.dumps({"status": "connected"}))
                    elif action == 'start':
                        if exercise in self.analyzers:
                            if not self.running:
                                self.current_analyzer = self.analyzers[exercise]
                                self.current_analyzer.reset_counters()  # Reset counters for new exercise
                                self.running = True
                                asyncio.create_task(self.process_frames())
                                await websocket.send(json.dumps({"status": "started", "exercise": exercise}))
                            else:
                                await websocket.send(json.dumps({"status": "already_running"}))
                        else:
                            await websocket.send(json.dumps({"error": "Invalid exercise"}))
                    elif action == 'stop':
                        if self.running:
                            self.running = False
                            await websocket.send(json.dumps({"status": "stopped"}))
                        else:
                            await websocket.send(json.dumps({"status": "not_running"}))
                    else:
                        logger.warning(f"Unknown action: {action}")
                        await websocket.send(json.dumps({"error": "Unknown action"}))
                except json.JSONDecodeError:
                    logger.error("Invalid JSON received")
                    await websocket.send(json.dumps({"error": "Invalid request format"}))
        except websockets.ConnectionClosed:
            logger.info(f"Client disconnected: {websocket.remote_address}")
        finally:
            self.clients.remove(websocket)

    def start_server(self, host='localhost', port=8000):
        """Start the WebSocket server."""
        def run_event_loop(loop):
            asyncio.set_event_loop(loop)
            loop.run_forever()

        self.event_loop = asyncio.new_event_loop()
        server_thread = threading.Thread(
            target=run_event_loop,
            args=(self.event_loop,),
            daemon=True
        )
        server_thread.start()

        asyncio.run_coroutine_threadsafe(self._start_websocket_server(host, port), self.event_loop)
        logger.info(f"WebSocket server started on ws://{host}:{port}")

    async def _start_websocket_server(self, host, port):
        self.server = await websockets.serve(
            partial(self.websocket_handler),
            host,
            port,
            ping_interval=30
        )

    def stop_server(self):
        """Stop the WebSocket server."""
        self.running = False
        if self.server:
            self.server.close()
            asyncio.run_coroutine_threadsafe(self.server.wait_closed(), self.event_loop)
            self.event_loop.call_soon_threadsafe(self.event_loop.stop)
            logger.info("WebSocket server stopped")

if __name__ == "__main__":
    server = VideoServer()
    server.start_server()
    try:
        while True:
            time.sleep(1)  # Keep main thread alive
    except KeyboardInterrupt:
        server.stop_server()

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from sklearn.preprocessing import StandardScaler, LabelEncoder
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
import cv2
import tensorflow as tf
import threading
import base64
import os
import numpy as np
import asyncio
import json
from collections import deque
import time
import websockets
import base64
import time
from typing import Dict, List
import joblib
from tensorflow.keras.models import load_model
import logging
import mediapipe as mp
import numpy as np
import pandas as pd
import asyncio
import threading
import time
import base64
import logging
import websockets
from functools import partial

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


class SquatAnalyzer:
    def __init__(self, model_path = r"C:\Users\Lenovo\Desktop\FYYP_MID EVAL\Website_PhysioVision\PhysioVision\Backend_Vision\models_vision\best_squat_model.keras", scaler_path = r"C:\Users\Lenovo\Desktop\FYYP_MID EVAL\Website_PhysioVision\PhysioVision\Backend_Vision\models_vision\preprocessed_data_scaler.joblib", label_encoder_path = r"C:\Users\Lenovo\Desktop\FYYP_MID EVAL\Website_PhysioVision\PhysioVision\Backend_Vision\models_vision\preprocessed_data_label_encoder.joblib" , window_size=30):
        """Initialize the squat analyzer with trained model and preprocessing tools"""
        # Load model and preprocessing tools
        self.clients = set()
        self.running = False
        self.event_loop = None
        self.server = None
        self.capture = None
        self.detector_thread = None

        self.model = tf.keras.models.load_model(model_path)
        self.scaler = joblib.load(scaler_path)
        self.label_encoder = joblib.load(label_encoder_path)
        self.window_size = window_size
        
        # Initialize MediaPipe Pose
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5,
            model_complexity=1
        )
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        
        # Create buffer for storing features
        self.features_buffer = deque(maxlen=window_size)
        
        # Store current prediction and confidence
        self.current_prediction = None
        self.prediction_confidence = 0.0
        self.last_predictions = deque(maxlen=5)  # Store last 5 predictions for smoothing
        
        
        # Feature names for the processed angles
        self.feature_names = [
            'left_knee_angle', 'right_knee_angle', 
            'left_hip_angle', 'right_hip_angle',
            'torso_vertical_angle', 'head_torso_angle',
            'knee_distance_normalized', 'ankle_distance_normalized',
            'left_squat_depth', 'right_squat_depth',
            'left_knee_angle_velocity', 'right_knee_angle_velocity',
            'left_hip_angle_velocity', 'right_hip_angle_velocity',
            'torso_vertical_angle_velocity', 'head_torso_angle_velocity',
            'knee_distance_normalized_velocity', 'ankle_distance_normalized_velocity',
            'left_squat_depth_velocity', 'right_squat_depth_velocity'
        ]
        
        # Keypoint mapping from MediaPipe to our preprocessing format
        self.keypoint_map = {
            'NOSE': self.mp_pose.PoseLandmark.NOSE,
            'LEFT_SHOULDER': self.mp_pose.PoseLandmark.LEFT_SHOULDER,
            'RIGHT_SHOULDER': self.mp_pose.PoseLandmark.RIGHT_SHOULDER,
            'LEFT_HIP': self.mp_pose.PoseLandmark.LEFT_HIP,
            'RIGHT_HIP': self.mp_pose.PoseLandmark.RIGHT_HIP,
            'LEFT_KNEE': self.mp_pose.PoseLandmark.LEFT_KNEE,
            'RIGHT_KNEE': self.mp_pose.PoseLandmark.RIGHT_KNEE,
            'LEFT_ANKLE': self.mp_pose.PoseLandmark.LEFT_ANKLE,
            'RIGHT_ANKLE': self.mp_pose.PoseLandmark.RIGHT_ANKLE
        }
        
        # Error explanations
        self.error_explanations = {
            'bad_back_round': "Your back is rounding. Keep your spine neutral.",
            'bad_back_warp': "Your back is excessively arched. Maintain a neutral spine.",
            'bad_head': "Head position incorrect. Look slightly downward, keeping your neck aligned with your spine.",
            'bad_inner_thigh': "Knees collapsing inward. Keep knees aligned with toes.",
            'bad_shallow': "Squat is too shallow. Try to go deeper with proper form.",
            'bad_toe': "Foot positioning issue. Keep feet shoulder-width apart with toes slightly turned out.",
            'good': "Good form! Keep it up."
        }




        # Add rep counting variables
        self.rep_count = 0
        self.state = 'STANDING'  # Initial state
        self.depth_history = deque(maxlen=10)  # Store recent squat depths for dynamic thresholding
        self.min_depth = None  # Dynamic minimum depth (updated during exercise)
        self.max_depth = None  # Dynamic maximum depth (updated during exercise)
        self.depth_threshold_factor = 0.5  # Percentage of depth range to consider a state change

        # Add error occurrence tracking
        self.error_counts = {
            'bad_back_round': 0,
            'bad_back_warp': 0,
            'bad_head': 0,
            'bad_inner_thigh': 0,
            'bad_shallow': 0,
            'bad_toe': 0,
            'good': 0  # Track good form too
        }



        
        
        print("Squat Analyzer initialized successfully")

    def _landmarks_to_keypoints_dict(self, landmarks):
        """Convert MediaPipe landmarks to format compatible with preprocessing code"""
        keypoints = {}
        for name, landmark_id in self.keypoint_map.items():
            landmark = landmarks.landmark[landmark_id]
            keypoints[f'{name}_x'] = landmark.x
            keypoints[f'{name}_y'] = landmark.y
            keypoints[f'{name}_z'] = landmark.z
        return keypoints

    def _keypoints_dict_to_df(self, keypoints_dict):
        """Convert keypoints dictionary to DataFrame"""
        return pd.DataFrame([keypoints_dict])

    def _calculate_vector(self, df, point1, point2):
        """Calculate vector between two keypoints"""
        vec = np.zeros((len(df), 3))
        vec[:, 0] = df[f'{point2}_x'].values - df[f'{point1}_x'].values
        vec[:, 1] = df[f'{point2}_y'].values - df[f'{point1}_y'].values
        vec[:, 2] = df[f'{point2}_z'].values - df[f'{point1}_z'].values
        return vec

    def _normalize_vector(self, vec):
        """Normalize a vector to unit length"""
        magnitude = np.sqrt(np.sum(vec**2, axis=1))
        magnitude = np.where(magnitude == 0, 1e-10, magnitude)
        vec_normalized = vec / magnitude[:, np.newaxis]
        return vec_normalized

    def _angle_between_vectors(self, vec1, vec2):
        """Calculate the angle between two 3D vectors in degrees"""
        vec1_norm = self._normalize_vector(vec1)
        vec2_norm = self._normalize_vector(vec2)
        dot_product = np.sum(vec1_norm * vec2_norm, axis=1)
        dot_product = np.clip(dot_product, -1.0, 1.0)
        angles = np.degrees(np.arccos(dot_product))
        return angles

    def _extract_anatomical_angles(self, df):
        """Extract biomechanically relevant angles from keypoints"""
        angles_df = pd.DataFrame()
        
        # Calculate vectors
        shoulder_to_hip_left = self._calculate_vector(df, 'LEFT_SHOULDER', 'LEFT_HIP')
        shoulder_to_hip_right = self._calculate_vector(df, 'RIGHT_SHOULDER', 'RIGHT_HIP')
        hip_to_knee_left = self._calculate_vector(df, 'LEFT_HIP', 'LEFT_KNEE')
        hip_to_knee_right = self._calculate_vector(df, 'RIGHT_HIP', 'RIGHT_KNEE')
        knee_to_ankle_left = self._calculate_vector(df, 'LEFT_KNEE', 'LEFT_ANKLE')
        knee_to_ankle_right = self._calculate_vector(df, 'RIGHT_KNEE', 'RIGHT_ANKLE')
        nose_to_shoulder_mid = self._calculate_vector(df, 'NOSE', 'LEFT_SHOULDER')
        
        # Calculate angles
        angles_df['left_knee_angle'] = self._angle_between_vectors(hip_to_knee_left, knee_to_ankle_left)
        angles_df['right_knee_angle'] = self._angle_between_vectors(hip_to_knee_right, knee_to_ankle_right)
        angles_df['left_hip_angle'] = self._angle_between_vectors(shoulder_to_hip_left, hip_to_knee_left)
        angles_df['right_hip_angle'] = self._angle_between_vectors(shoulder_to_hip_right, hip_to_knee_right)
        
        # Back angles (relative to vertical)
        vertical = np.zeros_like(shoulder_to_hip_left)
        vertical[:, 1] = 1  # Y axis is usually vertical in pose estimation
        angles_df['torso_vertical_angle'] = self._angle_between_vectors(shoulder_to_hip_left, vertical)
        
        # Head angle relative to torso
        angles_df['head_torso_angle'] = self._angle_between_vectors(nose_to_shoulder_mid, shoulder_to_hip_left)
        
        # Knee distance (for detecting knee valgus/varus)
        hip_width = np.sqrt(
            (df['RIGHT_HIP_x'] - df['LEFT_HIP_x'])**2 + 
            (df['RIGHT_HIP_z'] - df['LEFT_HIP_z'])**2
        )
        knee_distance = np.sqrt(
            (df['RIGHT_KNEE_x'] - df['LEFT_KNEE_x'])**2 + 
            (df['RIGHT_KNEE_z'] - df['LEFT_KNEE_z'])**2
        )
        angles_df['knee_distance_normalized'] = knee_distance / hip_width
        
        # Foot positioning
        ankle_distance = np.sqrt(
            (df['RIGHT_ANKLE_x'] - df['LEFT_ANKLE_x'])**2 + 
            (df['RIGHT_ANKLE_z'] - df['LEFT_ANKLE_z'])**2
        )
        angles_df['ankle_distance_normalized'] = ankle_distance / hip_width
        
        # Squat depth - hip height relative to knee height
        angles_df['left_squat_depth'] = df['LEFT_HIP_y'] - df['LEFT_KNEE_y']
        angles_df['right_squat_depth'] = df['RIGHT_HIP_y'] - df['RIGHT_KNEE_y']
        
        return angles_df

    def _add_velocity_features(self, current_features, prev_features=None, fps=30):
        """Add velocity features based on frame-to-frame changes"""
        velocity_features = {}
        
        if prev_features is None:
            # No previous frame, set velocities to 0
            for column in current_features.index:
                velocity_features[f'{column}_velocity'] = 0
        else:
            # Calculate frame-to-frame changes
            for column in current_features.index:
                velocity_features[f'{column}_velocity'] = (current_features[column] - prev_features[column]) * fps
                
        # Combine with current features
        all_features = {}
        for column in current_features.index:
            all_features[column] = current_features[column]
        all_features.update(velocity_features)
        
        return pd.Series(all_features)

    def _process_frame(self, frame):
        """Process a single frame and extract features"""
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the image
        results = self.pose.process(image_rgb)
        
        # If no pose detected, return None
        if not results.pose_landmarks:
            return None, frame
        
        # Draw pose landmarks on the image
        annotated_image = frame.copy()
        self.mp_drawing.draw_landmarks(
            annotated_image,
            results.pose_landmarks,
            self.mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=self.mp_drawing_styles.get_default_pose_landmarks_style()
        )
        
        # Convert landmarks to keypoints dictionary
        keypoints_dict = self._landmarks_to_keypoints_dict(results.pose_landmarks)
        
        # Convert to DataFrame
        keypoints_df = self._keypoints_dict_to_df(keypoints_dict)
        
        # Extract angles
        angles_df = self._extract_anatomical_angles(keypoints_df)
        
        # Get the first row as a Series
        feature_series = angles_df.iloc[0]
        
        # Add velocity features if we have previous features
        if len(self.features_buffer) > 0:
            prev_features = self.features_buffer[-1]
            feature_series = self._add_velocity_features(feature_series, prev_features)
        else:
            # No previous features, add zero velocities
            feature_series = self._add_velocity_features(feature_series)
        
        # Add to buffer
        self.features_buffer.append(feature_series)
        
        return feature_series, annotated_image

    def _make_prediction(self):
        """Make a prediction using the current feature buffer"""
        if len(self.features_buffer) < self.window_size:
            return None, 0.0
        
        # Prepare features for model input
        features_array = np.array([feature[self.feature_names] for feature in self.features_buffer])
        
        # Apply scaler transformation (same as in training)
        normalized_features = self.scaler.transform(features_array)
        
        # Reshape for model input [batch_size=1, window_size, num_features]
        model_input = normalized_features.reshape(1, self.window_size, len(self.feature_names))
        
        # Get prediction
        prediction_probs = self.model.predict(model_input, verbose=0)[0]
        predicted_class_idx = np.argmax(prediction_probs)
        confidence = prediction_probs[predicted_class_idx]
        
        # Get class name
        predicted_class = self.label_encoder.classes_[predicted_class_idx]
        
        return predicted_class, confidence

    def _smooth_predictions(self, new_prediction, new_confidence):
        """Smooth predictions to avoid flickering"""
        if new_prediction is None:
            return self.current_prediction, self.prediction_confidence
        
        # Add to recent predictions
        self.last_predictions.append((new_prediction, new_confidence))
        
        # Count occurrences of each prediction
        prediction_counts = {}
        total_confidence = {}
        
        for pred, conf in self.last_predictions:
            if pred not in prediction_counts:
                prediction_counts[pred] = 0
                total_confidence[pred] = 0
            
            prediction_counts[pred] += 1
            total_confidence[pred] += conf
        
        # Get the most common prediction
        if prediction_counts:
            most_common = max(prediction_counts.items(), key=lambda x: x[1])[0]
            avg_confidence = total_confidence[most_common] / prediction_counts[most_common]
            return most_common, avg_confidence
        
        return None, 0.0



    def _update_rep_count(self, current_depth):
        """Update squat state and count reps based on squat depth"""
        # Use average of left and right squat depth for consistency
        avg_depth = (current_depth['left_squat_depth'] + current_depth['right_squat_depth']) / 2
        self.depth_history.append(avg_depth)

        # Update min and max depths dynamically
        if len(self.depth_history) == self.depth_history.maxlen:
            current_min = min(self.depth_history)
            current_max = max(self.depth_history)
            
            if self.min_depth is None or current_min < self.min_depth:
                self.min_depth = current_min
            if self.max_depth is None or current_max > self.max_depth:
                self.max_depth = current_max

            # Calculate dynamic threshold
            if self.min_depth is not None and self.max_depth is not None:
                depth_range = self.max_depth - self.min_depth
                threshold = self.min_depth + (depth_range * self.depth_threshold_factor)

                # State transitions
                if self.state == 'STANDING' and avg_depth < threshold:
                    self.state = 'SQUATTING'
                elif self.state == 'SQUATTING' and avg_depth > threshold:
                    self.state = 'STANDING'
                    self.rep_count += 1  # Count a rep when returning to standing

    def _update_error_counts(self, prediction):
        """Update the count of the current prediction/error"""
        if prediction in self.error_counts:
            self.error_counts[prediction] += 1



    
    def draw_feedback(self, image, prediction, confidence):
        h, w, _ = image.shape
        
        # Background for text (make it larger to fit rep counter)
        cv2.rectangle(image, (0, h-140), (w, h), (0, 0, 0), -1)
        
        # Draw form feedback (existing code)
        if prediction is None:
            cv2.putText(image, "Getting ready...", (10, h-100), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        else:
            # Determine text color based on prediction
            if prediction == 'good':
                text_color = (0, 255, 0)  # Green for good form
            else:
                text_color = (0, 0, 255)  # Red for errors
            
            # Add prediction and confidence
            cv2.putText(image, f"Form: {prediction}", (10, h-100), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
            cv2.putText(image, f"Confidence: {confidence:.2f}", (10, h-70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
            
            # Add explanation for errors
            if prediction in self.error_explanations:
                explanation = self.error_explanations[prediction]
                cv2.putText(image, explanation, (w//4, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, text_color, 2)





        # Add rep count display
        cv2.putText(image, f"Reps: {self.rep_count}", (10, h-40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        return image




    def generate_report(self):
        """Generate a report summarizing reps and error occurrences"""
        report = f"Squat Analysis Report - {time.strftime('%Y-%m-%d %H:%M:%S')}\n"
        report += "=" * 50 + "\n"
        report += f"Total Reps Performed: {self.rep_count}\n\n"
        report += "Form Analysis:\n"
        report += "-" * 20 + "\n"
        
        total_frames_with_prediction = sum(self.error_counts.values())
        if total_frames_with_prediction > 0:
            for error, count in self.error_counts.items():
                percentage = (count / total_frames_with_prediction) * 100
                explanation = self.error_explanations.get(error, "No explanation available")
                report += f"{error}: {count} occurrences ({percentage:.1f}%)\n"
                report += f"  - {explanation}\n"
        else:
            report += "No form predictions recorded.\n"
        
        report += "=" * 50
        return report

    def reset_counters(self):
        """Reset rep count and error counts"""
        self.rep_count = 0
        self.state = 'STANDING'
        self.depth_history.clear()
        self.min_depth = None
        self.max_depth = None
        for error in self.error_counts:
            self.error_counts[error] = 0
        print("Counters reset")




        


    def rescale_frame(self, frame, scale_percent=50):
        """
        Rescale the input frame to improve processing speed.
        
        Args:
            frame: Input frame to be rescaled
            scale_percent: Percentage of original size (default: 50%)
            
        Returns:
            Rescaled frame
        """
        width = int(frame.shape[1] * scale_percent / 100)
        height = int(frame.shape[0] * scale_percent / 100)
        dim = (width, height)
        
        # Resize image
        resized = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
        return resized



   


if __name__ == "__main__":
    # Create analyzer instance
    squatanalyzer = SquatAnalyzer()

    try:
        # Start WebSocket server
        start_server(host='localhost', port=8765)

        # Keep the main thread alive
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        analyzer.stop_server()
        

In [ ]:

        
    async def process_video(self, input_source=0):
        cap = cv2.VideoCapture(input_source)
        # we can remove this if we want to keep counters after every time the camera is opened. 
        self.reset_counters()
        if not cap.isOpened():
            logger.error(f"Error: Could not open video source {input_source}")
            return

        processing_times = []

        try:
           
            while self.running and cap.isOpened():
                start_time = time.time()

                success, frame = cap.read()
                if not success:
                    logger.info("End of video")
                    break

                # Process the frame
                features, annotated_frame = self._process_frame(frame)

                # If no pose detected
                if features is None:
                    await self._broadcast({
                        "type": "frame",
                        "data": frame_base64,
                        "prediction": None,
                        "confidence": None,
                        "rep_count": self.rep_count
                    })
                    await asyncio.sleep(0.03)
                    continue

                # Make prediction if enough frames collected
                if len(self.features_buffer) >= self.window_size:
                    new_prediction, new_confidence = self._make_prediction()
                    self.current_prediction, self.prediction_confidence = self._smooth_predictions(
                        new_prediction, new_confidence)
                    if self.current_prediction is not None:
                        self._update_error_counts(self.current_prediction)

                # Update rep count with current features
                self._update_rep_count(features)

                # Encode frame as base64
                if annotated_frame is not None:
                    _, buffer = cv2.imencode('.jpg', annotated_frame)
                    frame_base64 = base64.b64encode(buffer).decode('utf-8')

                    await self._broadcast({
                        "type": "frame",
                        "data": frame_base64,
                        "prediction": self.current_prediction,
                        "confidence": self.prediction_confidence,
                        "rep_count": self.rep_count
                    })

                # Measure processing time
                processing_time = time.time() - start_time
                processing_times.append(processing_time)

                # Maintain FPS at ~30
                await asyncio.sleep(max(0, 0.03 - processing_time))

        except Exception as e:
            logger.error(f"Error during video processing: {e}")
        finally:
            cap.release()
            logger.info("Video processing stopped")


                # Print report at the end
        print("\n" + self.generate_report())
        with open('report.txt', 'w') as f: f.write(self.generate_report())

                

    async def websocket_handler(self, websocket):
        """Handle WebSocket connections."""
        self.clients.add(websocket)
        logger.info(f"New client connected: {websocket.remote_address}")
        try:
            async for message in websocket:
                try:
                    data = json.loads(message)
                    action = data.get('action')
                    logger.info(f"Received action: {action}")

                    if action == 'connect':
                        await websocket.send(json.dumps({"status": "connected"}))
                    elif action == 'start_squats':
                        if not self.running:
                            self.running = True
                            asyncio.create_task(self.start_processing())
                            await websocket.send(json.dumps({"status": "started"}))
                        else:
                            await websocket.send(json.dumps({"status": "already_running"}))
                    elif action == 'stop':
                        if self.running:
                            self.running = False
                            await websocket.send(json.dumps({"status": "stopped"}))
                        else:
                            await websocket.send(json.dumps({"status": "not_running"}))
                    else:
                        logger.warning(f"Unknown action: {action}")
                        await websocket.send(json.dumps({"error": "Unknown action"}))
                except json.JSONDecodeError:
                    logger.error("Invalid JSON received")
                    await websocket.send(json.dumps({"error": "Invalid request format"}))
                except Exception as e:
                    logger.error(f"Error handling message: {str(e)}")
                    await websocket.send(json.dumps({"error": f"Internal server error: {str(e)}"}))
        except websockets.exceptions.ConnectionClosed:
            logger.info(f"Client disconnected: {websocket.remote_address}")
        except Exception as e:
            logger.error(f"Unexpected error in connection handler: {str(e)}")
        finally:
            self.clients.remove(websocket)
            logger.info(f"Client removed: {websocket.remote_address}")

    
    async def start_processing(self):
        """Start processing video frames and streaming via WebSocket."""
        self.running = True
        await self.process_video(input_source=0)


    async def _broadcast(self, message):
        """Broadcast a message to all connected clients."""
        if not self.clients:
            return
            
        message_json = json.dumps(message)
        await asyncio.gather(
            *[client.send(message_json) for client in self.clients],
            return_exceptions=True
        )
    
    def broadcast_message(self, message):
        """Broadcast a message to all connected clients from any thread."""
        if self.event_loop and self.clients:
            asyncio.run_coroutine_threadsafe(self._broadcast(message), self.event_loop)
            
    def stop_processing(self):
        """Stop processing video frames."""
        self.running = False
        logger.info("Stopping processing")
        # Wait for the detector thread to finish
        if self.detector_thread and self.detector_thread.is_alive():
            self.detector_thread.join(timeout=5.0)
        # Camera will be released in the process_frames method's finally block



    def start_server(self, host='localhost', port=8765):
        """Start the WebSocket server."""
        def run_event_loop(loop):
            asyncio.set_event_loop(loop)
            loop.run_forever()

        # Create a new event loop for the WebSocket server
        self.event_loop = asyncio.new_event_loop()

        # Start the event loop in a separate thread
        server_thread = threading.Thread(
            target=run_event_loop,
            args=(self.event_loop,),
            daemon=True
        )
        server_thread.start()

        # Schedule the WebSocket server to start
        asyncio.run_coroutine_threadsafe(self._start_websocket_server(host, port), self.event_loop)

        logger.info(f"WebSocket server started on ws://{host}:{port}/ws")


async def _start_websocket_server(self, host, port):
        """Start the WebSocket server in the event loop."""
        self.server = await websockets.serve(
            partial(self.websocket_handler),  # Bind self to the method
            host,
            port,
            ping_interval=30
        )

    def stop_server(self):
        """Stop the WebSocket server."""
        if self.server:
            self.server.close()
            asyncio.run_coroutine_threadsafe(self.server.wait_closed(), self.event_loop)
            self.event_loop.call_soon_threadsafe(self.event_loop.stop)
            logger.info("WebSocket server stopped")

In [ ]:
async def process_video(self, frame):
        """Process a single frame and return data to broadcast."""
        # Process the frame with MediaPipe Pose
        results = self.pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        annotated_frame = frame.copy()  # Create a copy to annotate

        if results.pose_landmarks:
            self.mp_drawing.draw_landmarks(annotated_frame, results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS)
            errors = self.check_warrior_pose(results.pose_landmarks.landmark)

            # Update frame count and recording logic
            self.frame_count += 1
            if self.frame_count > self.delay_frames and not self.recording:
                self.recording = True
                self.start_frame = self.frame_count
            if self.recording and (self.frame_count - self.start_frame) <= self.record_frames:
                if not errors:
                    self.report["good_form_frames"] += 1
                for error in errors:
                    if error not in self.report["error_counts"]:
                        self.report["error_counts"][error] = 0
                    self.report["error_counts"][error] += 1

            # Display errors or "Correct Form" on the frame
            if errors:
                for i, error in enumerate(errors):
                    cv2.putText(annotated_frame, error, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(annotated_frame, "Correct Form", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Encode frame as base64 and return data
        frame_base64 = self._encode_frame(annotated_frame)
        return {
            "type": "frame",
            "data": frame_base64,
            "good_form_frames": self.report["good_form_frames"],
            "error_counts": self.report["error_counts"],
            "recording": self.recording,
            "frame_count": self.frame_count - self.start_frame if self.recording else 0
        }

    def _encode_frame(self, frame):
        """Encode frame as base64."""
        _, buffer = cv2.imencode('.jpg', frame)
        return base64.b64encode(buffer).decode('utf-8')

In [ ]:
class WarriorPoseAnalyzer:
    def __init__(self, record_seconds=10, fps=30):
        # Initialize MediaPipe Pose
        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils
        self.pose = self.mp_pose.Pose()

        # Define adjustable thresholds
        self.THRESHOLDS = {
            "front_knee_angle": (70, 120),
            "back_leg_angle": (150, 180),
            "hip_orientation": (0, 30),
            "torso_angle": (0, 100),
            "arm_angle": (160, 180),
            "shoulder_hip_alignment": (0, 40)
        }

        # Recording settings
        self.fps = fps
        self.delay_frames = 3 * fps  # 3 seconds delay
        self.record_frames = record_seconds * fps  # Number of frames to record
        self.frame_count = 0
        self.recording = False
        self.report = {
            "good_form_frames": 0,
            "error_counts": defaultdict(int)  # Tracks each error's frequency
        }

    def calculate_angle(self, p1, p2, p3):
        """Calculate the angle between three points in degrees."""
        a = np.array(p1)
        b = np.array(p2)
        c = np.array(p3)
        ba = a - b
        bc = c - b
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle) * 180 / np.pi
        return angle

    def check_warrior_pose(self, landmarks):
        """Analyze Warrior II pose and return top 3 errors."""
        errors = []

        l_hip = [landmarks[self.mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[self.mp_pose.PoseLandmark.LEFT_HIP].y]
        r_hip = [landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP].y]
        l_knee = [landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE].y]
        r_knee = [landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE].y]
        l_ankle = [landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE].y]
        r_ankle = [landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE].y]
        l_shoulder = [landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER].y]
        r_shoulder = [landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
        l_elbow = [landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW].x, landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW].y]
        r_elbow = [landmarks[self.mp_pose.PoseLandmark.RIGHT_ELBOW].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_ELBOW].y]
        l_wrist = [landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST].x, landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST].y]
        r_wrist = [landmarks[self.mp_pose.PoseLandmark.RIGHT_WRIST].x, landmarks[self.mp_pose.PoseLandmark.RIGHT_WRIST].y]

        left_knee_angle = self.calculate_angle(l_hip, l_knee, l_ankle)
        right_knee_angle = self.calculate_angle(r_hip, r_knee, r_ankle)
        
        if left_knee_angle < right_knee_angle:
            front_hip, front_knee, front_ankle = l_hip, l_knee, l_ankle
            back_hip, back_knee, back_ankle = r_hip, r_knee, r_ankle
            front_knee_angle = left_knee_angle
            back_leg_angle = right_knee_angle
            hip_angle = self.calculate_angle(l_hip, r_hip, [r_hip[0] + 1, r_hip[1]])
        else:
            front_hip, front_knee, front_ankle = r_hip, r_knee, r_ankle
            back_hip, back_knee, back_ankle = l_hip, l_knee, l_ankle
            front_knee_angle = right_knee_angle
            back_leg_angle = left_knee_angle
            hip_angle = self.calculate_angle(r_hip, l_hip, [l_hip[0] - 1, l_hip[1]])

        torso_angle = self.calculate_angle(
            [(l_hip[0] + r_hip[0]) / 2, (l_hip[1] + r_hip[1]) / 2],
            [(l_shoulder[0] + r_shoulder[0]) / 2, (l_shoulder[1] + r_shoulder[1]) / 2],
            [0, 1]
        )
        l_arm_angle = self.calculate_angle(l_shoulder, l_elbow, l_wrist)
        r_arm_angle = self.calculate_angle(r_shoulder, r_elbow, r_wrist)
        shoulder_hip_angle = self.calculate_angle(l_shoulder, r_shoulder, r_hip)

        if not self.THRESHOLDS["front_knee_angle"][0] <= front_knee_angle <= self.THRESHOLDS["front_knee_angle"][1]:
            errors.append("Bend your front knee more" if front_knee_angle > 100 else "Straighten your front knee slightly")
        if not self.THRESHOLDS["back_leg_angle"][0] <= back_leg_angle <= self.THRESHOLDS["back_leg_angle"][1]:
            errors.append("Straighten your back leg" if back_leg_angle < 160 else "Relax your back leg slightly")
        
        if not self.THRESHOLDS["hip_orientation"][0] <= hip_angle <= self.THRESHOLDS["hip_orientation"][1]:
            if front_hip == r_hip:
                if r_hip[1] > l_hip[1]:
                    errors.append("Level your hips; right hip is too high")
                else:
                    errors.append("Level your hips; left hip is too high")
            else:
                if l_hip[1] > r_hip[1]:
                    errors.append("Level your hips; left hip is too high")
                else:
                    errors.append("Level your hips; right hip is too high")

        if not self.THRESHOLDS["arm_angle"][0] <= l_arm_angle <= self.THRESHOLDS["arm_angle"][1] or \
           not self.THRESHOLDS["arm_angle"][0] <= r_arm_angle <= self.THRESHOLDS["arm_angle"][1]:
            errors.append("Raise your arms to shoulder level" if l_arm_angle < 170 or r_arm_angle < 170 else "Extend your arms fully")

        return errors[:3]

    def generate_report(self):
        """Generate and print an exercise report."""
        total_recorded_frames = self.record_frames
        good_form_seconds = self.report["good_form_frames"] / self.fps
        total_seconds = total_recorded_frames / self.fps

        print("\n--- Warrior II Exercise Report ---")
        print(f"Total Recorded Time: {total_seconds:.2f} seconds")
        print(f"Good Form Duration: {good_form_seconds:.2f} seconds ({(good_form_seconds / total_seconds) * 100:.1f}%)")
        print("Errors Detected:")
        if self.report["error_counts"]:
            for error, count in self.report["error_counts"].items():
                error_seconds = count / self.fps
                print(f"  - '{error}': {count} frames ({error_seconds:.2f} seconds, {(count / total_recorded_frames) * 100:.1f}%)")
        else:
            print("  - No errors detected!")
        print("--------------------------------\n")


    async def process_video(self, frame):
        """Process a single frame and return data to broadcast."""
        # Process the frame with MediaPipe Pose
        results = self.pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        annotated_frame = frame.copy()  # Create a copy to annotate

        if results.pose_landmarks:
            self.mp_drawing.draw_landmarks(annotated_frame, results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS)
            errors = self.check_warrior_pose(results.pose_landmarks.landmark)

            # Update frame count and recording logic
            self.frame_count += 1
            if self.frame_count > self.delay_frames and not self.recording:
                self.recording = True
                self.start_frame = self.frame_count
            if self.recording and (self.frame_count - self.start_frame) <= self.record_frames:
                if not errors:
                    self.report["good_form_frames"] += 1
                for error in errors:
                    if error not in self.report["error_counts"]:
                        self.report["error_counts"][error] = 0
                    self.report["error_counts"][error] += 1

            # Display errors or "Correct Form" on the frame
            if errors:
                for i, error in enumerate(errors):
                    cv2.putText(annotated_frame, error, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(annotated_frame, "Correct Form", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Encode frame as base64 and return data
        frame_base64 = self._encode_frame(annotated_frame)
        return {
            "type": "frame",
            "data": frame_base64,
            "good_form_frames": self.report["good_form_frames"],
            "error_counts": self.report["error_counts"],
            "recording": self.recording,
            "frame_count": self.frame_count - self.start_frame if self.recording else 0
        }

    def _encode_frame(self, frame):
        """Encode frame as base64."""
        _, buffer = cv2.imencode('.jpg', frame)
        return base64.b64encode(buffer).decode('utf-8')

    def __del__(self):
        self.pose.close()
